In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("merged.csv")

In [3]:
df.head()

,일시,태양방위각,태양고도각,발전량태그(단일),발전량태그,기온,강수량,풍속,풍향,습도,...,일사합성3,일사합성4,일사합성5,일사합성6,일사합성7,일사합성8,일사합성9,1시간평균 미세먼지농도(㎍/㎥),발전량(Target),일사량_제곱
0,2017-01-01 08:00:00,9.054183,305.880929,1,20171,0.5,0,0.6,70,97.0,...,0.070000,0.070000,0.070000,0.070000,0.070000,0.070000,0.070000,21.0,0.02,0.0049
1,2017-01-01 09:00:00,18.291215,316.520616,1,20171,1.5,0,0.3,0,94.0,...,0.350000,0.350000,0.350000,0.350000,0.350000,0.350000,0.350000,21.0,1.43,0.1225
2,2017-01-01 10:00:00,25.717324,329.151211,2,20172,2.8,0,0.7,250,87.0,...,0.803611,0.623862,0.565775,0.572804,0.601640,0.587263,0.568490,21.0,4.31,0.4761
3,2017-01-01 11:00:00,30.626463,343.864418,3,20173,4.7,0,0.8,270,78.0,...,1.139555,1.219951,1.144290,1.229677,1.159019,1.151897,1.146153,21.0,9.22,1.6384
4,2017-01-01 12:00:00,32.359053,0.000000,4,20174,6.7,0,1.0,230,68.0,...,1.528914,1.536110,1.551077,1.511959,1.497501,1.477924,1.436003,21.0,14.42,2.6569


In [4]:
df.tail()

,일시,태양방위각,태양고도각,발전량태그(단일),발전량태그,기온,강수량,풍속,풍향,습도,...,일사합성3,일사합성4,일사합성5,일사합성6,일사합성7,일사합성8,일사합성9,1시간평균 미세먼지농도(㎍/㎥),발전량(Target),일사량_제곱
32079,2023-11-30 14:00:00,26.757026,31.442280,4,20234,3.3,0,3.1,50,47.0,...,2.040513,2.048985,1.980999,2.032285,2.008790,1.998683,1.974334,18.0,34.820550,4.5796
32080,2023-11-30 15:00:00,19.213961,44.230567,4,20234,2.7,0,4.3,50,50.0,...,1.509679,1.505286,1.511529,1.516687,1.498087,1.505205,1.456877,21.0,26.061120,2.6569
32081,2023-11-30 16:00:00,9.863307,54.964711,2,20232,2.4,0,2.4,20,54.0,...,0.801175,0.813638,0.710169,0.781964,0.760058,0.748237,0.728912,16.0,11.787986,0.7225
32082,2023-11-30 17:00:00,-0.712510,64.129908,1,20231,2.0,0,1.6,50,59.0,...,0.230000,0.230000,0.230000,0.230000,0.230000,0.230000,0.230000,6.0,4.309163,0.0529
32083,2023-11-30 18:00:00,-12.098027,72.283349,1,20231,1.5,0,1.8,50,65.0,...,0.080000,0.080000,0.080000,0.080000,0.080000,0.080000,0.080000,NaN,1.989735,0.0064


In [5]:
features = ["태양방위각", "태양고도각", "기온", "습도", "일사", "전운량", "1시간평균 미세먼지농도(㎍/㎥)", "일사량_제곱"]
target = "발전량(Target)"

In [6]:
X = df[features].values
y = df[target].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

X_train_torch = torch.tensor(X_train, dtype=torch.float32)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32)
y_test_torch = torch.tensor(y_test, dtype=torch.float32)
y_test_actual = scaler_y.inverse_transform(y_test)

In [7]:
y_train_actual = scaler_y.inverse_transform(y_train)
y_test_actual  = scaler_y.inverse_transform(y_test)

# Check maxima
print("Max of y_train (original scale):", y_train_actual.max())
print("Max of y_test  (original scale):", y_test_actual.max())

Max of y_train (original scale): 49.526795
Max of y_test  (original scale): 49.867033


In [15]:
import numpy as np

# if y_test_actual is shape (n,1), flatten it first
y_flat = y_test_actual.flatten()  

# find the index of its maximum
max_idx = np.argmax(y_flat)   

# pull out the value
max_val = y_flat[max_idx]      

print(f"Max occurs at sample {max_idx}, with value {max_val:.4f}")


Max occurs at sample 5733, with value 49.8670
